In [72]:
#todo
#statistics of returns
#regression (?)
#improve charts (titles etc..)
#import financials, possibile in a new "tab"
#import twitter scraping for sentimetn
#additional tab with portfolio (?)

env: ALPHAVANTAGE_API_KEY=NTN3D6P6LIMZ6A2Q#alpha advantage api key
Dash app running on http://127.0.0.1:8050/


In [2]:
#graph imports
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
#data imports
import pandas as pd
import numpy as np
import os
import pandas_datareader.data as web
from datetime import datetime as dt
from bs4 import BeautifulSoup
import requests
import sys
################################################################################
%env ALPHAVANTAGE_API_KEY = NTN3D6P6LIMZ6A2Q#alpha advantage api key
################################################################################


from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='YOUR_API_KEY',output_format='pandas', indexing_type='date')

################################################################################
#adjust value function
################################################################################
def adjust(date, close, adj_close, in_col, rounding=4):
    '''
    If using forex or Crypto - Change the rounding accordingly!
    '''
    try:
        factor = adj_close / close
        return round(in_col * factor, rounding)
    except ZeroDivisionError:
        print('WARNING: DIRTY DATA >> {} Close: {} | Adj Close {} | in_col: {}'.format(date, close, adj_close, in_col))
        return 0
    
    
def drawText():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H2("Text"),
                ], style={'textAlign': 'center'}) 
            ])
        ),
    ])

app = JupyterDash(external_stylesheets=[dbc.themes.SLATE])

app.layout = html.Div([       

                    dcc.Input(id = 'stock-ticker', value = 'KO', type = 'text'),
                    html.Button(id = 'submit-ticker', n_clicks = 0, children = 'Submit'),
                    dcc.Store(id='stock-storage'),
                    html.Div(id = 'stock-name'),    
                    html.Div(id = 'stock-price'),    
                    html.Div([dcc.DatePickerRange(id="my-date-picker-range",
                                                    min_date_allowed=dt(2000, 1, 1),
                                                    max_date_allowed=dt.today(),
                                                    start_date = dt(2000, 1, 1),
                                                    end_date=dt.today(),
                                                    updatemode='bothdates')]),
                    html.Br(),
                    html.Div(id = 'candlestick-graph'),
                    html.Div(id = 'daily-returns-graph'),
                    html.Br(),
                    html.Div(id = 'cum-returns-graph'),
                    html.Div(id = 'daily-returns-time-graph'),    
])

@app.callback(Output(component_id='stock-storage', component_property='data'),
              Output(component_id='stock-name', component_property='children'),
              Output(component_id='stock-price', component_property='children'),
              Input('submit-ticker', 'n_clicks'),
              State('stock-ticker', 'value'))
def update_output(n_clicks,stock):
    df, meta_data = ts.get_daily_adjusted(stock, outputsize='full')
    df.sort_index(ascending=True, inplace = True)  
    
    url = "https://finance.yahoo.com/quote/{}".format(stock)

    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')

    price = soup.find('span', class_='Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)')
    name = soup.find('h1', class_ = 'D(ib) Fz(18px)')
    
    #get company BS
    
    #understand how to use environmental variable
    url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={}&apikey=NTN3D6P6LIMZ6A2Q'.format(stock)
    r = requests.get(url)
    data = r.json()

    #Annual Reports  
    dfAR = pd.json_normalize(data, record_path = 'annualReports')
    dfAR = dfAR.set_index(dfAR['fiscalDateEnding'])
    dfAR.drop(["fiscalDateEnding"], axis = 1, inplace = True)
    dfAR = dfAR.transpose()
    dfAR.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)
    for i in list(dfAR.keys()):
        for j in list(dfAR[i].index):
            if dfAR[i][j].isnumeric() or type(dfAR[i][j]) == int or type(dfAR[i][j]) == float :
                dfAR[i][j] = float(dfAR[i][j])/1000000000
            else:
                dfAR[i][j] = dfAR[i][j]

    #Quarterly Reports            
    dfQR = pd.json_normalize(data, record_path = 'quarterlyReports')
    dfQR = dfQR.set_index(dfQR['fiscalDateEnding'])
    dfQR.drop(["fiscalDateEnding"], axis = 1, inplace = True)
    dfQR = dfQR.transpose()
    dfQR.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)
    for i in list(dfQR.keys()):
        for j in list(dfQR[i].index):
            if dfQR[i][j].isnumeric() or type(dfQR[i][j]) == int or type(dfQR[i][j]) == float :
                dfQR[i][j] = float(dfQR[i][j])/1000000000
            else:
                dfQR[i][j] = dfQR[i][j]
    
    
    #convet back df to json? how to display df as a table?
    
    
    
    return df.to_json(), html.H2(name.text), html.H2("Last price: {}".format(price.text))

@app.callback(Output(component_id='daily-returns-graph' , component_property= 'children'),
              Output(component_id='candlestick-graph', component_property= 'children'),
              Output(component_id='cum-returns-graph' , component_property= 'children'),
              Output(component_id='daily-returns-time-graph',component_property='children'),
              Output(component_id='my-date-picker-range', component_property = 'start_date'),
               Output(component_id='my-date-picker-range', component_property = 'end_date'),
              Input('stock-storage','data') )
def update_chart(stock):
    if stock is not None:
        df = pd.read_json(stock)

        df['adj close'] = df['5. adjusted close']
        df['adj open'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['1. open'])
        df['adj high'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['2. high'])
        df['adj low'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['3. low'])
    
    
        #calculate daily returns
        df['daily_return'] = df['adj close'].pct_change()
        df['log_returns'] = (np.log(df['adj close'] /    df['adj close'].shift(-1)))
        # calculate cumluative return
        df['cumluative_return'] = np.exp(np.log1p(df['daily_return']).cumsum())*10000
    
        daily_returns = dcc.Graph(#id='daily-returns-graph',
                                        figure=px.histogram(df, x="daily_return",  title = 'Daily returns distribution'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Returns",
                                            yaxis_title="Frequency",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )   
        candlestick = dcc.Graph(#id='candlestick-graph',
                                        figure=go.Figure(data=[go.Candlestick(x=df.index,
                                                        open=df['adj open'],
                                                        high=df['adj high'],
                                                        low=df['adj low'],
                                                        close=df['adj close'])]
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        cum_returns = dcc.Graph(#id='cum-returns-graph',
                                        figure=px.line(df, x=df.index,y="cumluative_return", title = 'Cumulative returns'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Value of portfolio",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        daily_returns_time = dcc.Graph(#id='daily-returns-time-graph',
                                        figure=px.line(df, x=df.index,y="daily_return", title = 'Daily returns over time'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Returns",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )            
        date_picker = dcc.DatePickerRange(id = 'my-date-picker-range',
                                min_date_allowed=pd.to_datetime(df.index).min(),
                                max_date_allowed=pd.to_datetime(df.index).max(),
                                start_date=pd.to_datetime(df.index).min(),
                                end_date=pd.to_datetime(df.index).max(),
                                updatemode='bothdates')
    
        return daily_returns, candlestick,  cum_returns, daily_returns_time, pd.to_datetime(df.index).min(), pd.to_datetime(df.index).max()
    else:
        return None, None, None, None, None, None


@app.callback(Output(component_id='daily-returns-graph' , component_property= 'children'),
              Output(component_id='candlestick-graph', component_property= 'children'),
              Output(component_id='cum-returns-graph' , component_property= 'children'),
              Output(component_id='daily-returns-time-graph',component_property='children'),
              Input('my-date-picker-range', 'start_date'),
              Input('my-date-picker-range', 'end_date'),
              Input('stock-storage','data') )
def update_chart_time(start, end, stock):
    if stock is not None and end is not None and start is not None:
        df = pd.read_json(stock)
        df = df[start:end]#verificare per fare entrare giorno corrente
        df['adj close'] = df['5. adjusted close']
        df['adj open'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['1. open'])
        df['adj high'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['2. high'])
        df['adj low'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['3. low'])


        #calculate daily returns
        df['daily_return'] = df['adj close'].pct_change()
        df['log_returns'] = (np.log(df['adj close'] /    df['adj close'].shift(-1)))
        # calculate cumluative return
        df['cumluative_return'] = np.exp(np.log1p(df['daily_return']).cumsum())*10000
    
        daily_returns = dcc.Graph(#id='daily-returns-graph',
                                        figure=px.histogram(df, x="daily_return",  title = 'Daily returns distribution'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Returns",
                                            yaxis_title="Frequency",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )   
        candlestick = dcc.Graph(#id='candlestick-graph',
                                        figure=go.Figure(data=[go.Candlestick(x=df.index,
                                                        open=df['adj open'],
                                                        high=df['adj high'],
                                                        low=df['adj low'],
                                                        close=df['adj close'])]
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        cum_returns = dcc.Graph(#id='cum-returns-graph',
                                        figure=px.line(df, x=df.index,y="cumluative_return", title = 'Cumulative returns'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Value of protfolio",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )     
        daily_returns_time = dcc.Graph(#id='daily-returns-time-graph',
                                        figure=px.line(df, x=df.index,y="daily_return", title = 'Daily returns over time'
                                        ).update_layout(
                                            template='plotly_dark',
                                            plot_bgcolor= 'rgba(0, 0, 0, 0)',
                                            paper_bgcolor= 'rgba(0, 0, 0, 0)',
                                            title = {'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                                            xaxis_title="Time",
                                            yaxis_title="Returns",
                                        ),
                                        config={
                                            'displayModeBar': False
                                        }
                                    )            
    
    
        return daily_returns, candlestick,  cum_returns, daily_returns_time
    else:
        return None,None,None,None


# Run app and display result inline in the notebook
app.run_server(mode="external", port = '8060' )
#mode="external", host="127.0.0.1", port=8060

env: ALPHAVANTAGE_API_KEY=NTN3D6P6LIMZ6A2Q#alpha advantage api key
Dash app running on http://127.0.0.1:8060/


In [12]:
df, meta_data = ts.get_daily_adjusted('SPOT', outputsize='full')
df.sort_index(ascending=True, inplace = True)  
df['adj close'] = df['5. adjusted close']
df['adj open'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['1. open'])
df['adj high'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['2. high'])
df['adj low'] = np.vectorize(adjust)(df.index.date, df['4. close'], df['5. adjusted close'], df['3. low'])
    
    
#calculate daily returns
df['daily_return'] = df['adj close'].pct_change()
# calculate cumluative return
df['cumluative_return'] = np.exp(np.log1p(df['daily_return']).cumsum())*10000

In [46]:
import requests, re
url = "https://finance.yahoo.com/quote/KO/financials?p=KO"


source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

test = soup.find('div', class_='Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)')
    

AttributeError: 'NoneType' object has no attribute 'text'

In [114]:
import requests
import re

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=IBM&apikey=NTN3D6P6LIMZ6A2Q'
r = requests.get(url)
data = r.json()

#Annual Reports  
dfAR = pd.json_normalize(data, record_path = 'annualReports')
dfAR = dfAR.set_index(dfAR['fiscalDateEnding'])
dfAR.drop(["fiscalDateEnding"], axis = 1, inplace = True)
dfAR = dfAR.transpose()
dfAR.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)
for i in list(dfAR.keys()):
    for j in list(dfAR[i].index):
        if dfAR[i][j].isnumeric() or type(dfAR[i][j]) == int or type(dfAR[i][j]) == float :
            dfAR[i][j] = float(dfAR[i][j])/1000000000
        else:
            dfAR[i][j] = dfAR[i][j]
            
#Quarterly Reports            
dfQR = pd.json_normalize(data, record_path = 'quarterlyReports')
dfQR = dfQR.set_index(dfQR['fiscalDateEnding'])
dfQR.drop(["fiscalDateEnding"], axis = 1, inplace = True)
dfQR = dfQR.transpose()
dfQR.rename ( index = lambda x: re.sub("([a-z])([A-Z])","\g<1> \g<2>",x).capitalize() , inplace = True)
for i in list(dfQR.keys()):
    for j in list(dfQR[i].index):
        if dfQR[i][j].isnumeric() or type(dfQR[i][j]) == int or type(dfQR[i][j]) == float :
            dfQR[i][j] = float(dfQR[i][j])/1000000000
        else:
            dfQR[i][j] = dfQR[i][j]

fiscalDateEnding,2021-03-31,2020-12-31,2020-09-30,2020-06-30,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-12-31,2016-09-30,2016-06-30
Reported currency,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
Total assets,148.629,155.971,154.128,154.2,153.403,152.186,149.62,154.652,130.926,123.382,121.99,121.622,125.285,125.356,121.636,120.495,117.495,117.47,115.606,118.056
Total current assets,34.038,39.165,39.845,39.953,38.931,38.42,38.121,77.517,52.705,49.146,48.257,46.795,49.122,49.735,44.742,45.013,42.889,43.888,41.433,43.524
Cash and cash equivalents at carrying value,10.531,13.212,14.393,12.041,11.218,8.172,10.087,45.399,17.134,11.379,11.563,10.741,11.949,11.972,10.915,11.696,10.096,7.826,9.039,10.017
Cash and short term investments,11.131,13.812,15.593,14.104,11.865,8.868,10.82,46.273,18.006,11.997,14.495,11.707,12.842,12.58,11.515,12.295,10.695,8.527,9.968,10.617
Inventory,1.828,1.839,1.949,1.869,1.786,1.619,1.712,1.745,1.771,1.682,1.893,1.742,1.753,1.583,1.711,1.604,1.609,1.553,1.729,1.685
Current net receivables,16.067,18.738,17.87,19.447,20.669,23.795,20.959,24.738,27.945,30.563,27.087,28.34,29.229,31.63,27.126,26.958,25.869,29.245,25.196,26.547
Total non current assets,112.465,115.864,112.339,112.325,112.562,112.878,109.277,74.908,65.902,68.067,67.024,67.621,65.516,69.446,64.832,63.942,63.625,66.971,64.851,64.926
Property plant equipment,9.452,10.04,9.958,9.709,9.626,10.01,10.063,10.202,10.675,10.792,10.949,11.024,11.278,11.116,11.057,10.903,10.865,10.83,11.104,11.092
Accumulated depreciation amortization ppe,22.817,23.136,22.474,21.957,21.464,22.018,21.608,21.641,22.017,21.668,21.535,21.209,21.497,21.215,20.88,20.388,19.816,19.303,19.738,19.044
